# 掩码语言模型训练实例

## Step1 导入相关包

In [2]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer

## Step2 加载数据集

In [3]:
ds = Dataset.load_from_disk("./wiki_cn_filtered/")

In [4]:
ds

Dataset({
    features: ['source', 'completion'],
    num_rows: 10000
})

In [5]:
ds[0]

{'source': 'wikipedia.zh2307',
 'completion': "西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆，馆长是锺明善。\n历史\n2004年9月20日开始筹建，2013年4月8日正式建成开馆，位于西安交通大学兴庆校区陕西省西安市咸宁西路28号。建筑面积6,800平米，展厅面积4,500平米，馆藏文物4,900余件。包括历代艺术文物馆、碑石书法馆、西部农民画馆、邢良坤陶瓷艺术馆、陕西秦腔博物馆和书画展厅共五馆一厅。\n营业时间\n* 周一至周六：上午九点至十二点，下午一点至五点\n* 周日闭馆"}

## Step3 数据集处理

In [6]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

def process_func(examples):
    return tokenizer(examples["completion"], max_length=384, truncation=True)

In [7]:
tokenized_ds = ds.map(process_func, batched=True, remove_columns=ds.column_names)
tokenized_ds

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 10000
})

In [8]:
from torch.utils.data import DataLoader

dl = DataLoader(tokenized_ds, batch_size=2, collate_fn=DataCollatorForLanguageModeling(tokenizer, mlm=True, mlm_probability=0.15))

In [9]:
next(enumerate(dl))

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


(0,
 {'input_ids': tensor([[  101,  6205,  2128,   769,  6858,  1920,   103,  1300,   103,  7667,
           8020, 13135,   112,  9064, 12095,  8731,   103,  8181,  8736, 10553,
           8021,  3221,   103,  2429,   103,   754,  6205,   103,   103,   103,
           1920,  2110,  4638,  1300,  4289,  7667,  8024,  7667,  7270,  3221,
           7247,  3209,  1587,   511,  1325,  1380,  8258,  2399,   130,  3299,
           8113,  3189,   103,  1993,  5040,  2456,  8024,  8138,  2399,   125,
           3299,   129,  3189,  3633,  2466,  2456,  2768,  2458,  7667,  8024,
            855,   103,  6205,  2128,   769,  6858,  1920,  2110,  1069,  2412,
           3413,  1277,   103,  6205,  4689,  6205,  2128,  2356,  1496,  2123,
           6205,  6662,  8143,  1384,   511,  2456,  5029,  7481,  4916,   127,
            117,  8280,   103,  5101,   103,  2245,  1324,  7481, 16901,   125,
            103,  8195,  2398,  5101,  8024,  7667,  5966,  3152,  4289,   125,
            117,  2848

In [10]:
tokenizer.mask_token, tokenizer.mask_token_id

('[MASK]', 103)

## Step4 创建模型

In [11]:
model = AutoModelForMaskedLM.from_pretrained("hfl/chinese-macbert-base")

Some weights of the model checkpoint at hfl/chinese-macbert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Step5 配置训练参数

In [12]:
args = TrainingArguments(
    output_dir="./masked_lm",
    per_device_train_batch_size=32,
    logging_steps=10,
    num_train_epochs=1
)

## Step6 创建训练器

In [13]:
trainer = Trainer(
    args=args,
    model=model,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=True, mlm_probability=0.15)
)

## Step7 模型训练

In [14]:
trainer.train()

c:\Users\yuyao\miniconda3\envs\transformers\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/313 [00:00<?, ?it/s]

{'loss': 1.4273, 'learning_rate': 4.840255591054313e-05, 'epoch': 0.03}
{'loss': 1.3971, 'learning_rate': 4.680511182108626e-05, 'epoch': 0.06}
{'loss': 1.3217, 'learning_rate': 4.520766773162939e-05, 'epoch': 0.1}
{'loss': 1.4278, 'learning_rate': 4.361022364217253e-05, 'epoch': 0.13}
{'loss': 1.3457, 'learning_rate': 4.201277955271566e-05, 'epoch': 0.16}
{'loss': 1.4064, 'learning_rate': 4.041533546325879e-05, 'epoch': 0.19}
{'loss': 1.3346, 'learning_rate': 3.8817891373801916e-05, 'epoch': 0.22}
{'loss': 1.3364, 'learning_rate': 3.722044728434505e-05, 'epoch': 0.26}
{'loss': 1.3191, 'learning_rate': 3.562300319488818e-05, 'epoch': 0.29}
{'loss': 1.3108, 'learning_rate': 3.402555910543131e-05, 'epoch': 0.32}
{'loss': 1.2879, 'learning_rate': 3.242811501597444e-05, 'epoch': 0.35}
{'loss': 1.3391, 'learning_rate': 3.083067092651757e-05, 'epoch': 0.38}
{'loss': 1.3066, 'learning_rate': 2.9233226837060707e-05, 'epoch': 0.42}
{'loss': 1.3116, 'learning_rate': 2.7635782747603834e-05, 'epoc

TrainOutput(global_step=313, training_loss=1.3282560220541664, metrics={'train_runtime': 116.5904, 'train_samples_per_second': 85.77, 'train_steps_per_second': 2.685, 'train_loss': 1.3282560220541664, 'epoch': 1.0})

## Step8 模型推理

In [15]:
from transformers import pipeline

pipe = pipeline("fill-mask", model=model, tokenizer=tokenizer, device=0)

In [17]:
pipe("西安交通[MASK][MASK]博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆")

[[{'score': 0.9948391318321228,
   'token': 1920,
   'token_str': '大',
   'sequence': "[CLS] 西 安 交 通 大 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 0.0021944143809378147,
   'token': 2110,
   'token_str': '学',
   'sequence': "[CLS] 西 安 交 通 学 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 0.0005357894115149975,
   'token': 2339,
   'token_str': '工',
   'sequence': "[CLS] 西 安 交 通 工 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 0.0003660529328044504,
   'token': 7770,
   'token_str': '高',
   'sequence': "[CLS] 西 安 交 通 高 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"},
  {'score': 0.0003039983275812119,
   'token': 4906,
   'token_str': '科',
   'sequence': "[CLS] 西 安 交 通 科 [MASK] 博 物 馆 （ xi'an jiaotong university museum ） 是 一 座 位 于 西 安 交 通 大 学 的 博 物 馆 [SEP]"}],
 [{'score': 0.9971689581871

In [18]:
pipe("下面是一则[MASK][MASK]新闻。小编报道，近日，游戏产业发展的非常好！")

[[{'score': 0.09348516166210175,
   'token': 2031,
   'token_str': '娱',
   'sequence': '[CLS] 下 面 是 一 则 娱 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.043486643582582474,
   'token': 7028,
   'token_str': '重',
   'sequence': '[CLS] 下 面 是 一 则 重 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.04063192754983902,
   'token': 2207,
   'token_str': '小',
   'sequence': '[CLS] 下 面 是 一 则 小 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.03394673392176628,
   'token': 3173,
   'token_str': '新',
   'sequence': '[CLS] 下 面 是 一 则 新 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  {'score': 0.03209644556045532,
   'token': 5381,
   'token_str': '网',
   'sequence': '[CLS] 下 面 是 一 则 网 [MASK] 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'}],
 [{'score': 0.07695978879928589,
   'token': 4829,
   'token_str': '磅',
   'sequence': '[CLS] 下 面 是 一 则 [MASK] 磅 新 闻 。 小 编 报 道 ， 近 日 ， 游 戏 产 业 发 展 的 非 常 好 ！ [SEP]'},
  